# Redundancy Agent – Determining redundancy in one single signal

In this tutorial we generate a single signal containing redundant, correlated
information and supply it to the Redundancy Agent. The agent uses the redundancy in
the data vector together with some prior knowledge in order to calculate the best
consistent estimate with associated uncertainty, respecting all provided input
uncertainties. It rejects sensor values that may be erroneous. In this presented use
case the sensors do not directly measure the measurand, but the measurand is linked
to the sensor values by means of four linear equations. The fact that there are four
equations and not just one is the cause of the redundancy.

At the start of the main module we define all important parameters. For
detailed descriptions of the parameters please refer to the
[docstrings of the respective methods](https://agentmet4fof.readthedocs.io/en/latest/agentMET4FOF_agents.html#module-agentMET4FOF.agents.metrological_redundancy_agents).
Then we define the agents and start the network. The network and
the calculated results can be monitored in a browser at the
address http://127.0.0.1:8050/.


In [2]:
# %load redundancy_agent_one_signal.py
import numpy as np

from agentMET4FOF.agents.metrological_redundancy_agents import RedundancyAgent
from agentMET4FOF.agents.metrological_base_agents import (
    MetrologicalMonitorAgent,
)
from agentMET4FOF.agents.metrological_signal_agents import (
    MetrologicalGeneratorAgent,
)
from agentMET4FOF.network import AgentNetwork
from agentMET4FOF.streams.metrological_signal_streams import (
    MetrologicalMultiWaveGenerator,
)


def demonstrate_redundancy_agent_onesignal():
    batch_size = 10
    n_pr = batch_size
    sampling_frequency = 40
    signal_1_frequency = 6
    signal_2_frequency = 10
    signal_1_initial_phase = 1
    signal_2_initial_phase = 2
    signal_1_amplitude = 230
    signal_2_amplitude = 20
    exp_unc_abs = 0.2  # absolute expanded uncertainty
    probability_limit = 0.95

    # start agent network server
    agent_network = AgentNetwork(dashboard_modules=True)

    # Initialize signal generating class outside of agent framework.
    signal1 = MetrologicalMultiWaveGenerator(
        sfreq=sampling_frequency,
        freq_arr=np.array([signal_1_frequency, signal_2_frequency]),
        ampl_arr=np.array([signal_1_amplitude, signal_2_amplitude]),
        phase_ini_arr=np.array([signal_1_initial_phase, signal_2_initial_phase]),
        value_unc=exp_unc_abs,
    )
    # signal1.init_parameters(batch_size1=batch_size)

    # Data source agents.
    source_name1 = "Sensor1"  # signal1.metadata.metadata["device_id"]
    source_agent1 = agent_network.add_agent(
        name=source_name1, agentType=MetrologicalGeneratorAgent
    )
    source_agent1.init_parameters(signal=signal1, batch_size=batch_size)

    # Redundant data processing agent
    sensor_key_list = [source_name1]
    redundancy_name1 = "RedundancyAgent1"  # Name cannot contain spaces!!
    redundancy_agent1 = agent_network.add_agent(
        name=redundancy_name1, agentType=RedundancyAgent
    )
    redundancy_agent1.init_parameters(
        sensor_key_list=sensor_key_list,
        calc_type="lcss",
        n_pr=n_pr,
        problim=probability_limit,
    )
    # prior knowledge needed for redundant evaluation of the data
    redundancy_agent1.init_lcss_parameters(
        fsam=sampling_frequency,
        f1=signal_1_frequency,
        f2=signal_2_frequency,
        ampl_ratio=signal_1_amplitude / signal_2_amplitude,
        phi1=signal_1_initial_phase,
        phi2=signal_2_initial_phase,
    )

    # Initialize metrologically enabled plotting agent.(
    monitor_agent1 = agent_network.add_agent(
        name="MonitorAgent_SensorValues", agentType=MetrologicalMonitorAgent
    )
    monitor_agent2 = agent_network.add_agent(
        name="MonitorAgent_RedundantEstimate", agentType=MetrologicalMonitorAgent
    )

    # Bind agents.
    source_agent1.bind_output(monitor_agent1)
    source_agent1.bind_output(redundancy_agent1)
    redundancy_agent1.bind_output(monitor_agent2)

    # Set all agents states to "Running".
    agent_network.set_running_state()

    # Allow for shutting down the network after execution.
    return agent_network


if __name__ == "__main__":
    demonstrate_redundancy_agent_onesignal()

Starting NameServer...
Broadcast server running on 0.0.0.0:9091
NS running on 127.0.0.1:3333 (127.0.0.1)
URI = PYRO:Pyro.NameServer@127.0.0.1:3333

--------------------------------------------------------------
|                                                            |
| Your agent network is starting up. Open your browser and   |
| visit the agentMET4FOF dashboard on http://127.0.0.1:8050/ |
|                                                            |
--------------------------------------------------------------

INFO [2021-07-07 15:01:10.413429] (Sensor1): INITIALIZED
INFO [2021-07-07 15:01:10.450809] (RedundancyAgent1): INITIALIZED
INFO [2021-07-07 15:01:10.484041] (MonitorAgent_SensorValues): INITIALIZED
INFO [2021-07-07 15:01:10.524346] (MonitorAgent_RedundantEstimate): INITIALIZED
[2021-07-07 15:01:10.546007] (Sensor1): Connected output module: MonitorAgent_SensorValues
[2021-07-07 15:01:10.559178] (Sensor1): Connected output module: RedundancyAgent1
[2021-07-07 15:01:10.5

/home/ludwig10/code/agentMET4FOF/agentMET4FOF/metrological_streams.py:135: UserWarning:

No uncertainty generator function specified. Setting to default (value_unc = constant, time_unc = 0).



SET STATE:   Running
[2021-07-07 15:01:11.424478] (Sensor1): Pack time: 0.001013
[2021-07-07 15:01:11.428880] (Sensor1): Sending: [array([[  0.        ,   0.        , 115.89771535,   0.2       ]]), <time_series_metadata.scheme.MetaData object at 0x7f4d32e7be50>]
[2021-07-07 15:01:11.440180] (RedundancyAgent1): Received: {'from': 'Sensor1', 'data': array([[  0.        ,   0.        , 115.89771535,   0.2       ]]), 'metadata': <time_series_metadata.scheme.MetaData object at 0x7f4d32e917c0>, 'senderType': 'MetrologicalGeneratorAgent', 'channel': 'default'}
[2021-07-07 15:01:11.451151] (RedundancyAgent1): Buffer: {'Sensor1': array([[  0.        ,   0.        , 115.89771535,   0.2       ]])}
Buffer size is [2021-07-07 15:01:11.453337] (MonitorAgent_SensorValues): Received: {'from': 'Sensor1', 'data': array([[  0.        ,   0.        , 115.89771535,   0.2       ]]), 'metadata': <time_series_metadata.scheme.MetaData object at 0x7f4d32e933d0>, 'senderType': 'MetrologicalGeneratorAgent', 'chan

Buffer size is  [2021-07-07 15:01:15.428496] (RedundancyAgent1): Buffer: {'Sensor1': array([[ 0.00000000e+00,  0.00000000e+00,  1.15897715e+02,
         2.00000000e-01],
       [ 2.50000000e-02,  0.00000000e+00, -1.01535705e+02,
         2.00000000e-01],
       [ 5.00000000e-02,  0.00000000e+00, -2.14017298e+02,
         2.00000000e-01],
       [ 7.50000000e-02,  0.00000000e+00, -1.59767167e+02,
         2.00000000e-01],
       [ 1.00000000e-01,  0.00000000e+00,  4.97982699e+00,
         2.00000000e-01]])}
5[2021-07-07 15:01:15.429141] (MonitorAgent_SensorValues): Buffer: {'Sensor1': {'data': array([[ 0.00000000e+00,  0.00000000e+00,  1.15897715e+02,
         2.00000000e-01],
       [ 2.50000000e-02,  0.00000000e+00, -1.01535705e+02,
         2.00000000e-01],
       [ 5.00000000e-02,  0.00000000e+00, -2.14017298e+02,
         2.00000000e-01],
       [ 7.50000000e-02,  0.00000000e+00, -1.59767167e+02,
         2.00000000e-01],
       [ 1.00000000e-01,  0.00000000e+00,  4.97982699e+00,
 